In [1]:
import pandas as pd
import numpy as np

In [2]:
import may_function

In [3]:
from geopy import Nominatim
geolocator = Nominatim(user_agent="specify_may_app")
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
tqdm.pandas()

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
train = pd.read_csv("./train_dataset_train.zip", compression='zip')
test = pd.read_csv("./test_dataset_test.zip", compression='zip')
test['crop'] = -1
train = train[sorted(train.columns)]
test = test[sorted(test.columns)]

In [6]:
data = train.append(test).reset_index(drop=True)
del train, test

In [7]:
# Простой алгоритм извлечения одной координаты из полигона

data['geolist'] = data['.geo'].str.split(',')
data['longitude'] = data['geolist'].str[-2].replace("[^\d\.]", "", regex = True)
data['latitude'] = data['geolist'].str[-1].replace("[^\d\.]", "", regex = True)
data['location'] = data['latitude']+', '+data['longitude']

In [8]:
# Данный код будет работать несколько часов. Для примера, в итераторе первые десять строк. 
# Полная выгрузка в архиве data_all.zip

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
data['adress'] = data['location'][0:].progress_apply(geocode)

100%|███████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.10it/s]


In [8]:
data = may_function.downcast_dtypes(data)

In [10]:
data.to_pickle('./data_all.zip', compression='zip')

In [11]:
data.head(3)

,.geo,area,crop,id,nd_mean_2021-04-15,nd_mean_2021-04-16,nd_mean_2021-04-18,nd_mean_2021-04-19,nd_mean_2021-04-20,nd_mean_2021-04-22,...,nd_mean_2021-08-11,nd_mean_2021-08-12,nd_mean_2021-08-13,nd_mean_2021-08-23,nd_mean_2021-08-27,geolist,longitude,latitude,location,adress
0,"{""type"":""Polygon"",""coordinates"":[[[42.49107269...",20,3,3536,0.187150,0.072846,0.262300,0.261778,0.063134,0.062981,...,0.000000,0.658016,0.126832,0.034858,0.614770,"[{""type"":""Polygon"", ""coordinates"":[[[42.491072...",42.49107269973438,51.41403574496151,"51.41403574496151, 42.49107269973438","(Тоетьяковский территориальный отдел, Борисогл..."
1,"{""type"":""Polygon"",""coordinates"":[[[50.25545110...",45,4,3739,0.245285,0.332928,0.349133,0.287182,0.064671,0.097138,...,0.618475,0.601685,0.174914,0.000000,0.179612,"[{""type"":""Polygon"", ""coordinates"":[[[50.255451...",50.25545110440393,56.60539225586957,"56.60539225586957, 50.25545110440393","(Кугунур, Кугунурское сельское поселение, Балт..."
2,"{""type"":""Polygon"",""coordinates"":[[[37.99698998...",28,2,1294,0.006391,0.013679,0.000000,0.007055,0.000000,0.120804,...,0.172080,0.000000,0.090607,0.000000,0.054127,"[{""type"":""Polygon"", ""coordinates"":[[[37.996989...",37.99698998166838,53.83590426425984,"53.83590426425984, 37.99698998166838","(Дедиловское сельское поселение, Киреевский ра..."
